In [1]:
# can be a way better snippet of code :') 
from IPython.display import display
import os
os.chdir('./../')
exp = './experiment_abstract/'
from experiment_abstract.dataset import Dictionary, VQAFeatureDataset
dictionary = Dictionary.load_from_file(exp+'data/dictionary.pkl')
train_dset = VQAFeatureDataset('val', dictionary, dataroot=exp+'data/')
import pandas as pd
os.chdir('./visualization/')
data = pd.read_csv('./data/responses_abstract_val', sep='\t', header=None)
import numpy as np
pd.options.display.max_rows = None
import json
from collections import Counter

loading dictionary from ./experiment_abstract/data/dictionary.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/val_target.pkl


In [2]:
with open('./../experiment_abstract/data/abstract_v002_val2015_annotations.json') as file:
    original_questions = json.load(file)

In [3]:
original_questions['annotations'].sort(key= lambda x: int(x['question_id']))

In [4]:
data.shape, len(original_questions['annotations'])

((30000, 2049), 30000)

In [5]:
data[2048] = data[2048].astype(int) 
data.sort_values(by=2048, inplace=True)

In [6]:
types = [q['question_type'] for q in original_questions['annotations']] #' '.join(q['question_type'].split()[:2])
count_types = Counter(types)
check_type_order = sorted(list(set(count_types)))[::-1]
questions = [q['question'] for q in train_dset.entries]

outliers = []
for q in questions:
    add = True
    for t in check_type_order:
        if t in q.lower():
            add = False
            break
    if add:
        outliers.append(q.lower())
        
new_types = Counter([' '.join(w.strip().split()[:2]).strip() for w in outliers]).keys()
count_types.update(new_types)
check_type_order = sorted(list(set(count_types)))[::-1]

questions_types = []
for q in questions:
    add = True
    for t in check_type_order:
        if t in q.replace('  ',' ').lower():
            questions_types.append(t)
            add = False
            break
    if add:
        print(q)
len(questions_types)

30000

In [7]:
data['q_type'] = questions_types

In [8]:
data.to_csv('data/abstract_q_types_features_val.csv', header=None, index=None)